In [1]:
#math and statistics.
from scipy import stats
import statistics
import math

#impot matplot and seaborn for viz.
import matplotlib.pyplot as plt
import seaborn as sns

#numpy and pandas library
import numpy as np
import pandas as pd

#dataset library
from pydataset import data
import env
import acquire


import warnings 
warnings.filterwarnings("ignore")

from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Goals
The goal of this report is to reduce churn in Telco. This project aims to discover features that may be related to churn, and use these discoveries to create a machine learning classification model that can predict if a customer is about to churn or not.

# Planning

1.Setup Github repo. 

2.Accquire the Data.

3.Clean the Data, import the data cleaning fuction. 

4.Explore the Train Dataset.

5.Train and Evalutate models.

6.Create a csv file with predictions. 

7.Report conclusions and summaries.

# Accquire the Data

In [11]:
df = acquire.get_telco_data()

Using cached csv


In [4]:
df.head(2)

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,One year,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,DSL,Month-to-month,Mailed check


# Cleaning the Data

In [5]:
# First, we will drop some columns that are not very useful. 
#customer_id
#payment_type_id
# contract_type
#internet_service_type_id

df.columns

Index(['payment_type_id', 'contract_type_id', 'internet_service_type_id',
       'customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'internet_service_type', 'contract_type',
       'payment_type'],
      dtype='object')

In [7]:
df = df.drop(columns = ['payment_type_id', 'contract_type_id', 'internet_service_type_id',
       'customer_id'])

In [10]:
df.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 7043 non-null   object 
 1   senior_citizen         7043 non-null   int64  
 2   partner                7043 non-null   object 
 3   dependents             7043 non-null   object 
 4   tenure                 7043 non-null   int64  
 5   phone_service          7043 non-null   object 
 6   multiple_lines         7043 non-null   object 
 7   online_security        7043 non-null   object 
 8   online_backup          7043 non-null   object 
 9   device_protection      7043 non-null   object 
 10  tech_support           7043 non-null   object 
 11  streaming_tv           7043 non-null   object 
 12  streaming_movies       7043 non-null   object 
 13  paperless_billing      7043 non-null   object 
 14  monthly_charges        7043 non-null   float64
 15  tota

In [22]:
def prep_telco_data(df):
    df = acquire.get_telco_data()
    # Drop duplicate columns
    df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], inplace=True)
       
    # Drop null values stored as whitespace    
    df['total_charges'] = df['total_charges'].str.strip()
    df = df[df.total_charges != '']
    
    # Convert to correct datatype
    df['total_charges'] = df.total_charges.astype(float)
    
    # Convert binary categorical variables to numeric
    df['gender_encoded'] = df.gender.map({'Female': 1, 'Male': 0})
    df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
    df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
    df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
    df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
    df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})
    
    # Get dummies for non-binary categorical variables
    dummy_df = pd.get_dummies(df[['multiple_lines', \
                              'online_security', \
                              'online_backup', \
                              'device_protection', \
                              'tech_support', \
                              'streaming_tv', \
                              'streaming_movies', \
                              'contract_type', \
                              'internet_service_type', \
                              'payment_type']], dummy_na=False, \
                              drop_first=True)
    
    # Concatenate dummy dataframe to original 
    df = pd.concat([df, dummy_df], axis=1)
    
    return df

In [23]:
df = prep_telco_data(df)

Using cached csv


In [24]:
df

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,streaming_tv_Yes,streaming_movies_No internet service,streaming_movies_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,...,1,0,0,1,0,0,0,0,0,1
1,Male,0,No,No,9,Yes,Yes,No,No,No,...,0,0,1,0,0,0,0,0,0,1
2,Male,0,No,No,4,Yes,No,No,No,Yes,...,0,0,0,0,0,1,0,0,1,0
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,...,1,0,1,0,0,1,0,0,1,0
4,Female,1,Yes,No,3,Yes,No,No,No,No,...,1,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,0,No,No,13,Yes,No,Yes,No,No,...,0,0,0,1,0,0,0,0,0,1
7039,Male,0,Yes,No,22,Yes,Yes,No,No,No,...,0,0,1,0,0,1,0,0,1,0
7040,Male,0,No,No,2,Yes,No,No,Yes,No,...,0,0,0,0,0,0,0,0,0,1
7041,Male,0,Yes,Yes,67,Yes,No,Yes,No,Yes,...,0,0,1,0,1,0,0,0,0,1


## Explore the Train Dataset

### Questions that you want to answer:

1. Are customers with Fiber internet has different churn rate compare to customers with DSL?

2. Are customers who pay manually more likely to churn than customers who pay automatically?

3. Do customers who churn have a higher average monthly spend than those who don't?

In [26]:
train, test = train_test_split(df, train_size=0.8, random_state=1349, stratify=df.churn)
# 70/30 train validate split
train, validate = train_test_split(train, train_size=0.7, random_state=1349, stratify=train.churn)

In [28]:
print(train.shape)
print(test.shape)
print(validate.shape)

(3937, 47)
(1407, 47)
(1688, 47)


In [38]:
# Assuming a confidence level of 95% 
alpha = 0.05


**1. Are customers with Fiber internet has different churn rate compare to customers with DSL?**


In [60]:
#train.crosstab(x = "internet_service_type", y ='churn')

pd.crosstab(train.internet_service_type, train.churn)

#internet_service = pd.crosstab(train.internet_service_type, train.churn, margins=True).reset_index()

churn,No,Yes
internet_service_type,,
DSL,1114,262
Fiber optic,1007,724
None,770,60


In [ ]:
#H0: DSL churn rate == Fiber internet churn rate
#H1: DSL churn rate != Fiber internet churn rate  DSL churn rate

In [103]:
dsl_churn = train[train.internet_service_type == 'DSL'].churn_encoded

fiber_churn = train[train.internet_service_type == 'Fiber optic'].churn_encoded

In [63]:
t, p = stats.ttest_ind(fiber_churn, dsl_churn)
t, p

(13.968838676133931, 4.723757615226429e-43)

In [66]:
if (p < alpha) & (t > 0):
    print("We reject the null hypothesis that DSL churn rate == Fiber internet churn rate")
else:
    print("We fail to reject the null hypothesis")

We reject the null hypothesis that DSL churn rate == Fiber internet churn rate


**2. Are customers who pay manually more likely to churn than customers who pay automatically?**

In [100]:
#H0: Churn rate: customers who pay manually  <= customers who pay automatically
#H1: Churn rate: customers who pay manually > customers who pay automatically

In [94]:
train.columns

Index(['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'total_charges', 'churn',
       'internet_service_type', 'contract_type', 'payment_type',
       'gender_encoded', 'partner_encoded', 'dependents_encoded',
       'phone_service_encoded', 'paperless_billing_encoded', 'churn_encoded',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'online_security_No internet service', 'online_security_Yes',
       'online_backup_No internet service', 'online_backup_Yes',
       'device_protection_No internet service', 'device_protection_Yes',
       'tech_support_No internet service', 'tech_support_Yes',
       'streaming_tv_No internet service', 'streaming_tv_Yes',
       'streaming_movies_No internet service', 'streaming_movies_Yes',
       'contract_type_

In [95]:
pd.crosstab(train.payment_type, train.churn)

churn,No,Yes
payment_type,,
Bank transfer (automatic),711,148
Credit card (automatic),727,131
Electronic check,706,591
Mailed check,747,176


In [96]:
train['autopay'] = pd.DataFrame(train['payment_type'].str.contains('auto'))
train['manual']  = pd.DataFrame(train['payment_type'].str.contains('check'))

In [97]:
autopay_churn = train[train['autopay']].churn_encoded

manual_churn = train[train['manual']].churn_encoded

In [102]:
t, p = stats.ttest_ind(manual_churn, autopay_churn)
t, p/2

(13.169448889001739, 4.302822709861193e-39)

In [101]:
if (p/2 < alpha) & (t > 0):
    print("We reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")

We reject the null hypothesis


**3. Do customers who churn have a higher average monthly spend than those who don't?**